# このNootBookの説明
Yotube動画のトランスクリプトを取得し、会話の整理と評価をおこなう手順を検証します。

In [31]:
%pip install langchain==0.3.14
%pip install langchain-openai==0.2.14
%pip install youtube-transcript-api
%pip install pytube


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
from langchain_community.document_loaders import YoutubeLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI

## 補足説明
ルートのディレクトリにある .envファイルを作成し、必要なシークレットを設定してください。
シークレットの情報はAzureポータルのAzure OpenAIのリソース → [キーとエンドポイント] から取得できます。
```
AZURE_OPENAI_API_KEY=xxxx
AZURE_OPENAI_ENDPOINT=xxxx
AZURE_OPENAI_GPT4_DEPLOYMENT_NAME=gpt-4o
```

In [33]:
import os

from dotenv import load_dotenv

load_dotenv(".env", override=True)

True

## 参考情報
Yotube動画のトランスクリプトを取得する
- https://python.langchain.com/docs/integrations/document_loaders/youtube_transcript/

In [ ]:
youtube_url= "<限定公開用URL>"

loader = YoutubeLoader.from_youtube_url(
    youtube_url,
    # add_video_info=True,
    language=["ja"])

# ドキュメントを読み込む
documents = loader.load()
print(documents[0].page_content)

花風を引いてますけど鼻あ花風を引いて ますけどあのも題なくえっと問題ない問題 ないというかうん元気で元気ですよ少し だけ疲れてますけどそうなんですなんかで も前回よりげししてないですねあそうあの あの僕も思いましたなんか前回少し ふっくらしたなと思いましたでも体重は別 に増えたわけじゃ なく健康的な感じじゃないですかお そうですね確かにあの70は超えてない です69.何ぼとかって時々あるんです けどはい70はいかないですね68と69 の間を行ったり来たりしてますそうです なんか1番減った時とそんなに今変わって ないのではいやっぱ食べてるか食べてない かでねやっぱゲストリドが変わるんだなっ てすいませんあ本当ですね面白いです ね思いましたこんなに変るって今見て思い ます結構変わりやすいのかもしれないです ねそうれでくと確かに顔に出やすい方とか いらっしゃいますよ ねうんそうです多分ま僕嘘つけないタイプ な分顔に出るだと思いますそっちそっちの ですね本体調とかも全部顔出てああでも あの分かりやすくてれはいいです確か に確か にたと最近のお食事のはいまだ遅れて なかったんですけど奥さんにあの分析の 結果を出したの でしたいんだ えっとこの12123って書いてあんです が13なのかは日日が書いてなかったので ちょっと分からない状態ではい3日間3 日間です12とその間にあった食事の中で 朝晩ピックアップしてあと23日っていう 感じ あのしましたわかりましたはいなのでまだ こうあれですねあの回復してすぐのところ のお食事なので野菜の量としてはいつもの 量ではないていう158になってるの でちょっとあれあそうなんですまだやっぱ 食べないとなんですね野菜は結構多くなっ たと思ったんですけどあそうですか1番 食べてる時に比べる1番食べてる時だと 多分200とか250とか行ってるかなと 思うけどあそうなんですねはいちょっと 控えてた時期も含めてって感じなのでああ 分かりましたなるほど今の倍食べて いただいても300ちょっとなのであの倍 食べても大丈夫 ですそんなに食べれ ない本当ですかなんか割とあのやり始める と400とか行く方とかもいらっしゃるん ですよどうやって増やすんですかねその 自分自炊をしないからあありがとうやどう しようなんかあれですかね結構野菜サラダ 高いじゃないですかあ

In [ ]:
doc = documents[0].page_content

llm = AzureChatOpenAI(
            openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            azure_deployment=os.getenv("AZURE_OPENAI_GPT4_DEPLOYMENT_NAME"),
            openai_api_version="2023-09-15-preview",
            temperature=0.7,
            max_tokens=1600
        )

summary_prompt_template = """
管理栄養士と食習慣を改善したいユーザーとの会話です。
2者の会話を分けたうえで、面談のポイントをまとめてください。また以下の観点で管理栄養士が面談を進行できているかを評価してください。
アウトプットは各項目に対してはtrue, falseで返してください。
各項目はコーチングスキルです。
会話を全て記載する必要はなく、該当する文章のみあわせて記載してください。
管理栄養士とユーザーの会話の比率を計算してください。

###項目
- フィードバック:事実を客観的にとらえ、ユーザーに事実を伝えている
- メタコミュニケーション:コミュニケーションそのものを確認・調整するためのコミュニケーション、
  面談がうまく行っているかの確認
- 目標設定:具体的な行動目標の設定ができているか、その行動目標に対してユーザーの了承を得ているか
- ビジュアライズ:達成を見えるようにすること。①〜⑤のどの種類のビジュアライズができているかも評価してください。
①フューチャー・ペーシング
②モデルになる人に置き換える（モデリング）
③ディソシエート / アソシエート
④リソース:うまくいくイメージのもとになる資源・材料のこと。過去にうまくいった経験が役に立つこと。過去のダイエット経験、良い行動をした際の体験
⑤間接法
- 具体的な提案:何をどしたらよいか理由とともに、どのシーン、どのタイミングでどんな行動をしたらよいか
- アクノレッジメント:相手の変化や成果に気づき、それを言語化してはっきり伝えること。
単にほめることではない。承認する、相手の存在を認める、相手に感謝する、信頼してまかせること。
-傾聴を妨げる要因（ブロッキング）がない:勝手な推測、先入観、思い込み、興味関心、同一視、評価・批判、競争意識

-ユーザーの返答を受容しているか
-ユーザーの気持ちを汲み取っているか
-ユーザーの気持ちを無視していないか

全体を通してよりよくするための改善点をまとめてください。
----
{text}
"""

summary_prompt = PromptTemplate.from_template(
    summary_prompt_template)

summary = summary_prompt | llm | StrOutputParser()

result = summary.invoke({"text":doc})
print(result)

### 面談のポイントまとめ

1. **ユーザーの状況把握**
   - ユーザーが風邪気味であるが元気であることを確認。
   - 食事記録をもとに、野菜摂取量や脂質の摂取状況を分析。
   - 運動状況（自転車通勤ができなくなったことやランニングを控えていること）を把握。

2. **具体的な課題の明確化**
   - 野菜摂取量が不足している点を認識し、目標量を提示。
   - 魚介類の摂取頻度が少ないことを指摘し、改善案を提案。
   - 脂質摂取量のバランスを取る必要がある点を指摘。
   - プロテインやレバー、野菜など、具体的な食品の提案。

3. **実行可能な提案**
   - 野菜摂取を増やすための具体的な方法（袋野菜を活用、重い野菜を選ぶ）。
   - 魚介類を取り入れるための提案（デリカップやエビ、イカなどの代替案）。
   - 間食に関する提案（スルメイカも塩分を注意しつつOK）。

4. **ユーザーの目標設定**
   - 目標体重と現状とのギャップを明確化し、食事の改善のみで達成可能であることを説明。
   - ユーザーに無理のない範囲で取り組める具体的な行動を提案。

5. **次回の計画**
   - 次回の面談日程を調整し、海外滞在を考慮したアドバイスを検討。

---

### コーチングスキルの評価

#### フィードバック: true
- 「脂質は増えすぎていないが、注意が必要」「野菜摂取量は不足している」など、事実を客観的にユーザーに伝えている。

#### メタコミュニケーション: true
- 面談の進行状況を確認し、ユーザーの理解度を確認する場面が複数あった。

#### 目標設定: true
- ユーザーの了承を得つつ、無理のない範囲で具体的な行動目標を設定している。

#### ビジュアライズ: true
- ①フューチャー・ペーシング: 「3ヶ月後に減量効果が期待できる」。
- ④リソース: 「過去の良い食事習慣や成功体験を活用」。
- 全体的にビジュアライズが適切に行われている。

#### 具体的な提案: true
- 野菜の選び方や間食の工夫など、行動に移しやすい提案をしている。

#### アクノレッジメント: true
- 「良い減量ペースですね」「脂質の減らし方が適切です」といった相手の努力を

### 最終評価のプロンプトをテストする

In [ ]:
doc = result

llm = AzureChatOpenAI(
            openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            azure_deployment=os.getenv("ZURE_OPENAI_GPT4_DEPLOYMENT_NAME"),
            openai_api_version="2023-09-15-preview",
            temperature=0.5
        )

evaluation_prompt_template = """
以下のユーザーと管理栄養士の会話を評価してください:
{text}
"""
    
evaluation_prompt = PromptTemplate.from_template(
    evaluation_prompt_template)

# チェーン
chain = evaluation_prompt | llm | StrOutputParser() 
result = chain.invoke({"text":doc})